In [10]:
import spacy
import random
import pandas as pd 
data = pd.read_csv(r'C:\Users\Vaishali\Desktop\New folder\qa_data.csv',encoding= 'unicode_escape') 
import nltk 
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer() 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

nlp = spacy.load('en')

stop_words = set(stopwords.words('english')) 
  
VERB_CODES = {'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'}
  
from nltk.corpus import stopwords 
import numpy as np 
topic=np.unique(data['Concept'] ) 

finaldata = data[["Concept", "Answer"]]          
finaldata.head(10) 

import speech_recognition as sr 
 
mic_name = "USB Device 0x46d:0x825: Audio (hw:1, 0)"
#Sample rate is how often values are recorded 
sample_rate = 48000
chunk_size = 2048
#Initialize the recognizer 
r = sr.Recognizer() 

def preprocess_sentences(text): 
  text = text.lower() 
  temp_sent =[] 
  words = nltk.word_tokenize(text) 
  tags = nltk.pos_tag(words) 
  for i, word in enumerate(words): 
      if tags[i][1] in VERB_CODES:  
          lemmatized = lemmatizer.lemmatize(word, 'v') 
      else: 
          lemmatized = lemmatizer.lemmatize(word) 
      if lemmatized not in stop_words and lemmatized.isalpha(): 
          temp_sent.append(lemmatized) 
          
  finalsent = ' '.join(temp_sent) 
  finalsent = finalsent.replace("n't", " not") 
  finalsent = finalsent.replace("'m", " am") 
  finalsent = finalsent.replace("'s", " is") 
  finalsent = finalsent.replace("'re", " are") 
  finalsent = finalsent.replace("'ll", " will") 
  finalsent = finalsent.replace("'ve", " have") 
  finalsent = finalsent.replace("'d", " would") 
  return finalsent 

finaldata["Answer_processed"]= finaldata["Answer"].apply(preprocess_sentences) 
finaldata.head()



,Concept,Answer,Answer_processed
0,OOPS,bject-Oriented Programming or OOPs refers to l...,programming oops refers language use object pr...
1,OOPS,it is a type of programming which is based on ...,type program base object rather function proce...
2,OOPS,"Inheritance, Encapsulation, Polymorphism, Data...",inheritance encapsulation polymorphism data ab...
3,Data structure,A data structure is a particular way of organi...,data structure particular way organize data co...
4,Data structure,"a data structure is a data organization, manag...",data structure data organization management st...


In [14]:

def check(Y,X):
    flag=0
    for i in X["Answer"]:
        main_doc = nlp(i)
        similar=main_doc.similarity(nlp(Y))
        if similar > 0.65:
            print("Valid answer")
            flag=1
            break 
    if flag == 0:
         print("Invalid answer")          
    
def chat():
    question=random.choices(topic)
    X=finaldata.loc[finaldata['Concept'] == question[0]]
    X_list=[]
    for i in X["Answer_processed"]:
        X_list.append(word_tokenize(i))  
    print("Tell me about %s" %question)
    mic_list = sr.Microphone.list_microphone_names() 
  
    for i, microphone_name in enumerate(mic_list): 
        if microphone_name == mic_name: 
            device_id = i 
    with sr.Microphone(device_index = 1, sample_rate = sample_rate,chunk_size = chunk_size) as source:  
        r.adjust_for_ambient_noise(source) 
        print("Say Something")
        #listens for the user's input 
        audio = r.listen(source) 
        text=""
          
        try: 
            text = r.recognize_google(audio) 
            print("you said: %s" %text) 
 
        except sr.UnknownValueError: 
            print("Google Speech Recognition could not understand audio") 
      
        except sr.RequestError as e: 
            print("Could not request results from Google")
    check(text,X)
        
for _ in range(4):
    main_doc=nlp("hello")
    Y="hai"
    main_doc.similarity(nlp(Y))
    chat()

E:\Anaconda\lib\site-packages\ipykernel_launcher.py:47: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.


Tell me about ['Linked List']
Say Something
you said: data structure in which the data stored in non contiguous memory allocation
Valid answer
Tell me about ['Linked List']


E:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  


Say Something
you said: meaning of linear data structure
Valid answer
Tell me about ['Data structure']
Say Something
you said: data structure of organising and managing the data
Invalid answer
Tell me about ['OOPS']
Say Something
you said: contact in which the new constitution encapsulation inheritance polymorphism in extraction
Valid answer
